### NOT WORKING NOW, SITE CHANGED

In [2]:
import requests
import re
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import time
#import geopy
#from geopy.distance import vincenty

Магический стриппер! (ﾉ◕ヮ◕)ﾉ*:･ﾟ✧

In [3]:
def html_stripper(text):
    return re.sub('<[^<]+?>', '', str(text))

Еще одна магическая штука, которую я нашел на просторах гугла - она как-то работает! Это пригодится нам потом, когда мы будем сортировать массив строк в числовом порядке.

In [4]:
numbers = re.compile(r'(\d+)')
def numericalSort(value):
    parts = numbers.split(value)
    parts[1::2] = map(int, parts[1::2])
    return parts

Поиграв немного с ВАО, я понял, что можно сделать оптимальное разбиение по цене (например, 6.5М рублей), и тогда в каждом запросе будет не более 840 квартир.

In [5]:
urls = ['http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D={}&engine_version=2&maxprice=6500000&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1',
            'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D={}&engine_version=2&minprice=6500001&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1']
dist = 2
page = 1
print(urls[1].format(dist, page))

http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=2&engine_version=2&minprice=6500001&offer_type=flat&p=1&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1


In [6]:
start_time = time.time()
links = []
# 56, ..., 71 - номера районов, соответствующих ВАО
for dist in range(56, 72):
    start_time_dist = time.time()
    last_len_links = len(links)
    print('DISTRICT {}'.format(dist))
    for url in urls:
        # Если по запросу выдается, скажем, 16 страниц, то если мы перескочим на 17, нас перенаправит
        # на первую. Переменная is_looped отвечает за то, чтобы мы не зациклились
        is_looped = 0
        page = 1
        while is_looped == 0:
            # тут мы просто собираем информацию обо всех объявлениях на странице
            # (при помощи пристального вглядывания в html-код и профессионального кодинга (͡° ͜ʖ ͡°))
            page_url =  url.format(dist, page)
            search_page = BeautifulSoup(requests.get(page_url).content, 'lxml')
            pages = search_page.findAll('div', attrs = {'class':"pager_pages"})
            pages = re.split('<span>|</span>', str(pages))
            # условие зацикливания
            if page != 1 and ('1' in pages or pages[0] == '[]'):
                print('Total pages: ', page - 1)
                print('Total urls: ', len(links) - last_len_links)
                print('Unique urls: ', len(list(set(links))) - last_len_links)
                is_looped = 1
            # если не зациклились, то достаем все ссылки и переходим к следующей странице
            else:
                flat_urls = search_page.findAll('div', attrs = {'ng-class':"{'serp-item_removed': offer.remove.state, 'serp-item_popup-opened': isPopupOpen}"})
                flat_urls = re.split('http://www.cian.ru/sale/flat/|/" ng-class="', str(flat_urls))
                cnt = 0
                for link in flat_urls:
                    if link.isdigit():
                        links.append(link)
                        cnt += 1
                print('Number of flat urls on the page {}: '.format(page), cnt)
                page += 1
    # нам нужны только уникальные ссылки - как видно из вывода, некоторые объявления повторяются
    links = list(set(links))
    print('Total unique flat urls for district {}: '.format(dist), len(links) - last_len_links)
    print('Time spent on district {}: {} minutes'.format(dist, (time.time() - start_time_dist) / 60))
print('Total time spent: {} munutes'.format((time.time() - start_time) / 60))
print('Total unique flat urls: {}'.format(len(links)))

DISTRICT 56
Number of flat urls on the page 1:  0
Total pages:  1
Total urls:  0
Unique urls:  0
Number of flat urls on the page 1:  0
Total pages:  1
Total urls:  0
Unique urls:  0
Total unique flat urls for district 56:  0
Time spent on district 56: 0.17894813617070515 minutes
DISTRICT 57
Number of flat urls on the page 1:  0
Total pages:  1
Total urls:  0
Unique urls:  0
Number of flat urls on the page 1:  0
Total pages:  1
Total urls:  0
Unique urls:  0
Total unique flat urls for district 57:  0
Time spent on district 57: 0.1728175441424052 minutes
DISTRICT 58
Number of flat urls on the page 1:  0
Total pages:  1
Total urls:  0
Unique urls:  0
Number of flat urls on the page 1:  0
Total pages:  1
Total urls:  0
Unique urls:  0
Total unique flat urls for district 58:  0
Time spent on district 58: 0.18163824876149495 minutes
DISTRICT 59
Number of flat urls on the page 1:  0
Total pages:  1
Total urls:  0
Unique urls:  0
Number of flat urls on the page 1:  0
Total pages:  1
Total urls

Ура, мы получили массив номеров ссылок! Отсортируем его, вспомнив про магическую функцию сравнения в самом начале.

In [7]:
links = sorted(links, key=numericalSort)

In [8]:
len(links)
links[:7]
links[-6:]

[]

In [9]:
np.savetxt('vao_links.csv', links, delimiter=',', header='link', comments='', fmt='%s')

Теперь займемся функциями, с помощью которых мы сможем доставать интересующую нас информацию со страничек. Начнем с

#### N – номер квартиры по порядку

In [10]:
def getN(link):
    return int(link)

(это было слишком просто)

#### Rooms – число комнат

In [11]:
def getRooms(flat_page):
    rooms = flat_page.find('div', attrs={'class':"object_descr_title"})
    rooms = re.split(' |-', str(rooms))
    rooms = ''.join([i for i in rooms if i.isdigit()])
    # если квартира многокомнатная (6+), то количества комнат в том месте, где мы ищем, не стоит, и
    # у нас вылетает ValueError. Искать это самое число непросто - на разных подобных страничках
    # оно может стоять в разных местах, может быть в тексте, а может вообще не существовать. Поэтому
    # я решил не запариваться и выводить в подобных случаях 6, благо таких случаев на весь ВАО 11
    try:
        return int(rooms)
    except ValueError:
        return 6

#### Price – цена

В цене на сайте стоят пробельчики, но мы их убираем

In [12]:
def getPrice(flat_page):
    price = flat_page.find('div', attrs={'class':'object_descr_price'})
    price = re.split('<div>|руб|\W', str(price))
    price = ''.join([i for i in price if i.isdigit()][-3:])
    return int(price)

#### Totsp – общая площадь квартиры, кв. м.

In [13]:
def getTotsp(flat_page):
    totsp = flat_page.find('table', attrs={'class': "object_descr_props flat sale"})
    totsp = html_stripper(totsp).replace('\n', '')
    totsp = re.split('Общая площадь:|\ха0м2', totsp)
    totsp = re.split('\xa0м2', totsp[1])
    # бывает, что некоторых данных на странице нет, как, например, общей площади квартиры.
    # В таких случаях при ValueError мы будем выводить -1 (такие значения в обычном случае все равно
    # нигде не встретятся) и хитро с этим справляться в самом конце
    try:
        return float(totsp[0].replace(',', '.'))
    except ValueError:
        return -1

#### Livesp – жилая площадь квартиры, кв. м.

In [14]:
def getLivesp(flat_page):
    livesp = flat_page.find('table', attrs={'class': "object_descr_props flat sale"})
    livesp = html_stripper(livesp).replace('\n', '')
    livesp = re.split('Жилая площадь:|\ха0м2', livesp)
    livesp = re.split('\xa0м2', livesp[1])
    try:
        return float(livesp[0].replace(',', '.'))
    except ValueError:
        return -1

#### Kitsp – площадь кухни, кв. м.

In [15]:
def getKitsp(flat_page):
    kitsp = flat_page.find('table', attrs={'class': "object_descr_props flat sale"})
    kitsp = html_stripper(kitsp).replace('\n', '')
    kitsp = re.split('Площадь кухни:|\ха0м2', kitsp)
    kitsp = re.split('\xa0м2', kitsp[1])
    try:
        return float(kitsp[0].replace(',', '.'))
    except ValueError:
        return -1

#### Dist – расстояние от центра в км. (можно по Пифагору)

In [21]:
def getDist(flat_page):
    # вытаскиваем координаты из карты
    coords = flat_page.find('div', attrs={'class':"map_info_button_extend"}).contents[1]
    coords = re.split('&amp|center=|%2C', str(coords))
    coords_list = []
    for item in coords:
        if item[0].isdigit():
            coords_list.append(item)
    flat_coords = (float(coords_list[0]), float(coords_list[1]))
    # загугленные координаты центра Москвы
    mos_cent_coords = (55.753559, 37.609218)
    try:
        # пользуемся библиотекой geopy
        return vincenty(flat_coords, mos_cent_coords).kilometers
    except ValueError:
        return -1

#### Metrdist – расстояние до метро в минутах

In [16]:
def getMetrdist(flat_page):
    metrdist = flat_page.find('span', attrs={'class': "object_item_metro_comment"})
    metrdist = html_stripper(metrdist).replace(' ', '')
    metrdist = metrdist.split('\n')
    dists = [int(i) for i in metrdist if i.isdigit()]
    try:
        return int(min(dists))
    except ValueError:
        return -1

#### Walk – 1 – если пешком от метро, 0 – если на транспорте

In [17]:
def getWalk(flat_page):
    getwalk = flat_page.find('span', attrs={'class': "object_item_metro_comment"})
    getwalk = html_stripper(getwalk).replace(' ', '')
    if 'пешк' in getwalk:
        return 1
    else:
        return 0

#### Brick – 1 – кирпичный/монолит/жб, 0 – другой

Вообще-то информация о стенах дома может быть в двух разных местах страницы, поэтому я проверяю оба

In [18]:
def getBrick(flat_page):
    getbrick = flat_page.find('table', attrs={'class': "bti__data"})
    getbrick = html_stripper(getbrick).replace('\n', '')
    getbrick = re.split('Материалы стен|Этажность', getbrick)
    if len(getbrick) != 1:
        if 'кирп' in getbrick[1] or 'мон' in getbrick[1] or 'жб' in getbrick[1]\
                               or'ж/б' in getbrick[1] or 'железобетон' in getbrick[1]:
            return 1
        else:
            return 0
    else:
        getbrick = flat_page.find('table', attrs={'class': "object_descr_props flat sale"})
        getbrick = html_stripper(getbrick).replace('\n', '')
        getbrick = re.split('Тип дома:|Тип продажи:', getbrick)
        try:
            if 'кирп' in getbrick[1] or 'мон' in getbrick[1] or 'жб' in getbrick[1]\
                               or'ж/б' in getbrick[1] or 'железобетон' in getbrick[1]:
                return 1
            else:
                return 0
        except ValueError:
            return -1

#### Tel – 1 – если есть, 0 – нет

In [19]:
def getTel(flat_page):
    gettel = flat_page.find('table', attrs={'class': "object_descr_props flat sale"})
    gettel = html_stripper(gettel).replace('\n', '')
    gettel = re.split('Телефон:|Вид из окна:', gettel)
    if len(gettel) != 3 or 'не' in gettel[1] or '–' in gettel[1]:
        return 0
    else:
        return 1

#### Bal – 1 – есть балкон/лоджия, 0 – нет

In [20]:
def getBal(flat_page):
    getbal = flat_page.find('table', attrs={'class': "object_descr_props flat sale"})
    getbal = html_stripper(getbal).replace('\n', '')
    getbal = re.split('Балкон:|Лифт:', getbal)
    if len(getbal) != 3 or 'не' in getbal[1] or '–' in getbal[1]:
        return 0
    else:
        return 1

#### Floor – номер этажа, на котором расположена квартира

In [21]:
def getFloor(flat_page):
    getfloor = flat_page.find('table', attrs={'class': "object_descr_props flat sale"})
    getfloor = html_stripper(getfloor).replace('\n', '')
    getfloor = re.split('Этаж:|\xa0', getfloor)
    try:
        return int(getfloor[1])
    except ValueError:
        return -1

#### Nfloors – всего этажей в доме

In [22]:
def getNfloors(flat_page):
    getnfloors = flat_page.find('table', attrs={'class': "object_descr_props flat sale"})
    getnfloors = html_stripper(getnfloors).replace('\n', '')
    getnfloors = re.split('/\xa0|Тип дома:', getnfloors)
    try:
        return int(getnfloors[1])
    except ValueError:
        return -1

#### New – 1 – первичный рынок, 0 - вторичка

In [23]:
def getNew(flat_page):
    getnew = flat_page.find('table', attrs={'class': "object_descr_props flat sale"})
    getnew = html_stripper(getnew).replace('\n', '')
    getnew = re.split('Тип дома:|Высота потолков:', getnew)
    if 'втор' in getnew[1]:
        return 0
    else:
        return 1

In [24]:
#links = pd.read_csv('vao_links.csv')
#links = list(np.array(links['link']).astype(str))

Ссылки есть, функции есть, мы готовы гулять по страничкам!

In [25]:
columns = ['N', 'Rooms', 'Price', 'Totsp', 'Livesp', 'Kitsp', 'Dist', 'Metrdist', 'Walk', 'Brick',
          'Tel', 'Bal', 'Floor', 'Nfloors', 'New']
df_final = pd.DataFrame(columns=columns)

In [32]:
def processing(links):
    n = len(links)
    # количество ошибок (а они встречаются)
    errors = 0
    for link in links[:]:
        flat_page_url = 'http://www.cian.ru/sale/flat/' + link + '/'
        print('\r Processing link {}, {} of {} ({}% completed, total errors: {})'.format(
                flat_page_url, n - len(links) + errors + 1, n,
                round((n - len(links) + errors + 1) / n * 100, 2), errors), end='')
        try:
            flat_page = BeautifulSoup(requests.get(flat_page_url).content, 'lxml')
            df_final.loc[len(df_final)] = [getN(link), getRooms(flat_page), getPrice(flat_page),
                                 getTotsp(flat_page), getLivesp(flat_page), getKitsp(flat_page),
                                 getDist(flat_page), getMetrdist(flat_page), getWalk(flat_page),
                                 getBrick(flat_page), getTel(flat_page), getBal(flat_page),
                                 getFloor(flat_page), getNfloors(flat_page), getNew(flat_page)]
            # если ссылка обработана без ошибок, удаляем ее из исходного массива
            links.remove(link)
        except OSError or MaxRetryError or ConnectionError or ConnectionResetError:
            errors += 1
    if links:
        print('\nSome links were not processed. Trying to process them again...')
        # после процесса обработки в массиве могут остаться только необработанные ссылки
        # (где были ошибки в связи с соединением и т.п.); значит, если он не пуст, они есть, и надо
        # повторить процедуру еще раз
        return processing(links)
    else:
        print('\nAll links were processed')

In [33]:
start_time = time.time()
processing(links)
print('Time spent: {} minutes'.format((time.time() - start_time) / 60))


All links were processed
Time spent: 1.666545867919922e-05 minutes


Отлично, все получилось!

In [34]:
len(df_final)

0

Некоторые ссылки задублировались?

In [35]:
len(df_final['N'].value_counts()[df_final['N'].value_counts() > 1])

0

In [37]:
df_final.head(16)
df_final.tail(5)

,N,Rooms,Price,Totsp,Livesp,Kitsp,Dist,Metrdist,Walk,Brick,Tel,Bal,Floor,Nfloors,New


Далее становится понятно, почему для пропущенного значения использовалось -1, а не nan: мы переводим большинство столбцов в int (зачем, например, номеру ссылки тип 'float'?..)

In [38]:
df_final[['N', 'Rooms', 'Price', 'Metrdist', 'Walk', 'Brick',
          'Tel', 'Bal', 'Floor', 'Nfloors', 'New']] = \
df_final[['N', 'Rooms', 'Price', 'Metrdist', 'Walk', 'Brick',
          'Tel', 'Bal', 'Floor', 'Nfloors', 'New']].astype(int)
df_final.head(8)

,N,Rooms,Price,Totsp,Livesp,Kitsp,Dist,Metrdist,Walk,Brick,Tel,Bal,Floor,Nfloors,New


И вот теперь заменяем -1 на nan:

In [39]:
df_final.replace(-1, np.nan, inplace=True)
df_final.head(16)

,N,Rooms,Price,Totsp,Livesp,Kitsp,Dist,Metrdist,Walk,Brick,Tel,Bal,Floor,Nfloors,New


Видно, что один из столбцов типа int (Nfloors) перешел обратно в float. Это связано с тем, что numpy.nan имеет тип float, значит, только этот столбец из всех целочисленных содержал пропуски. С этим ничего сделать не получится, поэтому оставим все как есть.
Итак, ответ готов!

In [40]:
df_final.to_csv('vao_flats.csv')

Итак, в данной части проекта с помощью парсинга сайта cian.ru были получены данные обо всех продающихся в ВАО квартирах. При желании эту процедуру можно проделать для других районов (а также метро и т.п.), выбрав соответствующий критерий для запросов во избежание потери информации (чтобы по каждому запросу выдавалось не больше 840 квартир).